block 1 to 6 is api integration

In [1]:
import pandas as pd


In [2]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import datetime
import secret

In [3]:
api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)

In [4]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=x5ugrx91yexerba1&v=3


In [7]:
data = kite.generate_session("pZT2M7vC9yM9wH6h89T70DNDQ2AjifMM", api_secret=api_secret)
kite.set_access_token(data["access_token"])
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)
print(data["access_token"])

capjCwAeE8DIFiOdQLDVLrC58W7GkY8L


CALCULATE ALL THE LEVELS OF THE STOCKS (SWING HIGH AND SWING LOW)

In [77]:
def swingTop_bottom(monthly_data):
    data=[]
    length = len(monthly_data)

# n = int(inpucollectiont())  this will be used for specifying the counter for swing top
    for i in range(2,length-2):
        if ((monthly_data['high'][i] > monthly_data['high'][i+1]) and (monthly_data['high'][i] > monthly_data['high'][i+2]) and (monthly_data['high'] [i]> monthly_data['high'][i-1]) and (monthly_data['high'][i] > monthly_data['high'][i-2])):
            data.append(float(monthly_data['high'][i]))
         
        if ((monthly_data['low'][i] < monthly_data['low'][i+1]) and (monthly_data['low'][i] < monthly_data['low'][i+2]) and (monthly_data['low'][i] < monthly_data['low'][i-1]) and (monthly_data['low'][i] < monthly_data['low'][i-2])):
            data.append(float(monthly_data['low'][i]))

    return data

FUNCTION FOR CALCULATING THE NUMBER OF TAPS

In [78]:
def filterdTop(swingList,filteredata,levelAllowance):
    swingLen = len(swingList)
    filterdataLen = len(filteredata)
    # check=[]
    data ={}
    for i in range(swingLen):
        price =swingList[i]
        # print(price)
        percentage = ((price/100)*levelAllowance)
        subPercentage = price - percentage
        addPercentage = price + percentage
        # print(subPercentage,addPercentage)
        key = price
        # check.append('END')
        if key not in data:
            data[key]=0
        for j in range(filterdataLen):
            if ((float(filteredata['open'][j]) <= addPercentage) and (float(filteredata['open'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'open',float(filteredata['open'][j])])
            elif ((float(filteredata['high'][j]) <= addPercentage) and (float(filteredata['high'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'high',float(filteredata['high'][j])])
            elif ((float(filteredata['low'][j]) <= addPercentage) and (float(filteredata['low'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'low',float(filteredata['low'][j])])
            elif ((float(filteredata['close'][j]) <= addPercentage) and (float(filteredata['close'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'close',float(filteredata['close'][j])])
    
    return data

levels percentage

In [79]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge
       

function for first short listing of the stocks on the basis of 0.5%

In [95]:


def first_shortlist(symbol,addpercentage,subpercentage,ohlcCompare,close_data):
    stock_dict ={}
    stock_dict[symbol]=(set(),close_data)
    st = list(ohlcCompare.keys())
    close_add_percentage, close_sub_percentage = percentage(close_data, 4)
    for levels in range(len(st)):
       
        if (subpercentage <= float(st[levels]) <= addpercentage):
            
            if levels + 2 < len(st) and levels - 2 >= 0:
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels -1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels -2]]))
                
            elif levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                 

            elif levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]])) 

            elif levels - 1 >= 0 and levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                 
                

            elif levels + 1 < len(st) and levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]]))
                

   
    if stock_dict[symbol][0]:
        return stock_dict
    
    

this function take out the levels and there taps and this function will run at the end of the week

In [ ]:
import json
watchlist ="nifty500"
timeframe = 'M'
instruments = kite.instruments("NSE")
nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
errorStockList=[]
levelTapping={}

for symbols in nifty50['Symbol']:
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)
        to_date = datetime.date.today()
        from_date = "2019-10-01"
        data = kite.historical_data(instrument_token, from_date, to_date, "day")
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS

        monthly_data = df.groupby('date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()

        st = swingTop_bottom(monthly_data) # important levels for each stock
        ohlcCompare = filterdTop(st,df,levelAllowance=0.15)
        levelTapping[symbols]=ohlcCompare
        # print(levelTapping)
        
    except StopIteration:
        errorStockList.append(symbols)
        continue
print(levelTapping)
jasonfile = {k: json.dumps(v) for k,v in levelTapping.items()}
df= pd.DataFrame(list(jasonfile.items()),columns=["stocks","levels"])
df.to_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\levelsTaps.csv",index=False)

This function will sort the stocks on the basis of intraday(0.5) or positional(1) and on the basis of taps in descending order

In [105]:
from datetime import date
import ast
today = date.today()
instruments = kite.instruments("NSE")
levelAllowance = 0.5
nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
leveltap = pd.read_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\levelsTaps.csv")
leveltap['levels'] = leveltap['levels'].apply(json.loads)
i=0
first_sort= {}
levelTapping ={}
errorStockList=[]
for symbols in nifty50['Symbol']:
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)      
        try:
            ltp = kite.ltp('NSE:'+symbols)
            close_data = ltp['NSE:'+symbols]['last_price']
        except:
            print('error')
        addpercentage,subpercentage = percentage(close_data,levelAllowance) #THIS VALUES REPRESENTS THE RANGE FROM LTP
        sortedlist = first_shortlist(symbols,addpercentage,subpercentage,leveltap.iloc[i]['levels'],close_data)
        i+=1
        if sortedlist != None:
            first_sort[instrument_token] = sortedlist
    except StopIteration:
        errorStockList.append(symbols)
        continue
  
flattened_data = []
for key,values in first_sort.items():
    for symbol,prices in values.items():
        price_set =  list(prices[0])
        flattened_data.append([key,symbol,price_set,prices[1]])
first_sorted_dataframe =  pd.DataFrame(flattened_data,columns=['token','symbol','levels','ltp'])
first_sorted_csv = first_sorted_dataframe.to_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv",index=False)
csvFile= pd.read_csv('E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv')

csvFile['levels'] = csvFile['levels'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
df = csvFile.explode('levels')
df[['level','taps']] = pd.DataFrame(df['levels'].tolist(), index = df.index)
df=df.drop(columns='levels')

df = df.sort_values(by='taps', ascending=False)
if levelAllowance==1:
    df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_positional_{today.strftime("%d_%m_%Y")}.csv',index=False)
else:
    df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_intraday_{today.strftime("%d_%m_%y")}_{today.strftime("%b_%d_%Y")}.csv',index=False)
    
print(errorStockList)
print(len(errorStockList))

['COCHINSHIP']
1
